# Sampling pose DKF trained on H3.6M

In [1]:
%matplotlib inline
import os
import addpaths
from load import loadDataset
from common import insert_junk_entries
import os
import numpy as np
from scipy.signal import convolve2d

# Stupid hack to make parameter loading actually work
# (ugh, this is super brittle code)
import sys
del sys.argv[1:]

# CONFIG_PATH = './chkpt-h36m/DKF_lr-8_0000e-04-vm-R-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-mlp-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-5000-uid-config.pkl'
# WEIGHT_PATH = './chkpt-h36m/DKF_lr-8_0000e-04-vm-R-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-mlp-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-5000-uid-EP375-params.npz'
# sys.argv.extend('-vm R -infm structured -ds 10 -dh 50'.split())

# CONFIG_PATH = './chkpt-h36m/DKF_lr-8_0000e-04-vm-L-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-mlp-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-5000-past-only-config.pkl'
# WEIGHT_PATH = './chkpt-h36m/DKF_lr-8_0000e-04-vm-L-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-mlp-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-5000-past-only-EP25-params.npz'
# sys.argv.extend('-vm L -infm structured -ds 10 -dh 50 -uid past-only'.split())

CONFIG_PATH = './chkpt-h36m/DKF_lr-8_0000e-04-vm-L-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-conditional-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-5000-past-only-cond-emis-config.pkl'
WEIGHT_PATH = './chkpt-h36m/DKF_lr-8_0000e-04-vm-L-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-conditional-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-5000-past-only-cond-emis-EP25-params.npz'
sys.argv.extend('-vm L -infm structured -ds 10 -dh 50 -uid past-only-cond-emis -etype conditional'.split())
sys.argv.extend(['-reload', WEIGHT_PATH, '-params', CONFIG_PATH])

from parse_args_dkf import params
from utils.misc import removeIfExists,createIfAbsent,mapPrint,saveHDF5,displayTime
from stinfmodel_fast.dkf import DKF
import stinfmodel_fast.learning as DKF_learn
import stinfmodel_fast.evaluate as DKF_evaluate

/home/sam/repos/structuredinference/expt-h36m/..


Using TensorFlow backend.
ERROR (theano.sandbox.cuda): ERROR: Not using GPU. Initialisation of device 1 failed:
Bad device number 1. Only 1 devices available.


Loading linear matrices
Loading linear matrices


Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 4007)


In [2]:
dataset = loadDataset()

Spawning pool


In [3]:
params['savedir']+='-h36m'
# createIfAbsent(params['savedir'])

# Add dataset and NADE parameters to "params" which will become part of the
# model
for k in ['dim_observations','data_type']:
    params[k] = dataset[k]
mapPrint('Options: ',params)
if params['use_nade']:
    params['data_type']='real_nade'

# Remove from params
removeIfExists('./NOSUCHFILE')
reloadFile = params.pop('reloadFile')
pfile=params.pop('paramFile')
# paramFile is set inside the BaseClass in theanomodels
# to point to the pickle file containing params"""
assert os.path.exists(pfile),pfile+' not found. Need paramfile'
print 'Reloading trained model from : ',reloadFile
print 'Assuming ',pfile,' corresponds to model'
dkf  = DKF(params, paramFile = pfile, reloadFile = reloadFile)

Options:     {
shuffle : False 
use_nade : False 
cov_explicit : False 
dataset :  
epochs : 2000 
seed : 1 
init_weight : 0.1 
reg_spec : _ 
expt_name : past-only-cond-emis 
use_prev_input : False 
reg_value : 0.05 
reloadFile : ./chkpt-h36m/DKF_lr-8_0000e-04-vm-L-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-conditional-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-5000-past-only-cond-emis-EP25-params.npz 
dim_stochastic : 10 
rnn_layers : 1 
transition_layers : 2 
lr : 0.0008 
reg_type : l2 
init_scheme : uniform 
replicate_K : 1 
optimizer : adam 
data_type : real 
use_generative_prior : False 
q_mlp_layers : 1 
maxout_stride : 4 
batch_size : 20 
savedir : ./chkpt-h36m 
forget_bias : -5.0 
inference_model : structured 
emission_layers : 2 
savefreq : 25 
rnn_size : 600 
paramFile : ./chkpt-h36m/DKF_lr-8_0000e-04-vm-L-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-conditional-previnp-False-ar-1_0000e+01-

/home/sam/.conda/envs/dkf/lib/python2.7/site-packages/theano/sandbox/cuda/type.py:186: UserWarning: Warning, Cannot compute test value: input 0 (newX) of Op GpuFromHost(newX) missing default value
  return theano.sandbox.cuda.basic_ops.GpuFromHost()(other)
/home/sam/.conda/envs/dkf/lib/python2.7/site-packages/theano/sandbox/cuda/type.py:186: UserWarning: Warning, Cannot compute test value: input 0 (newMask) of Op GpuFromHost(newMask) missing default value
  return theano.sandbox.cuda.basic_ops.GpuFromHost()(other)
/home/sam/repos/theanomodels/models/__init__.py:201: UserWarning: lr found in npWeights. No action taken
  warnings.warn(name+" found in npWeights. No action taken")
/home/sam/repos/theanomodels/models/__init__.py:201: UserWarning: anneal found in npWeights. No action taken
  warnings.warn(name+" found in npWeights. No action taken")
/home/sam/repos/theanomodels/models/__init__.py:201: UserWarning: update_ctr found in npWeights. No action taken
  warnings.warn(name+" found in

	<<Anneal = 1 in 10.0 param. updates>>
****** CREATING TRAINING FUNCTION*****
	<<Building with dropout:0.1>>
	<<In <_buildLSTM>>>
	<<In _LSTMlayer with dropout 0.1000>>


/home/sam/.conda/envs/dkf/lib/python2.7/site-packages/theano/sandbox/rng_mrg.py:62: UserWarning: Warning, Cannot compute test value: input 0 (A) of Op DotModulo(A, s, m, A2, s2, m2) missing default value
  o = DotModulo()(A_sym, s_sym, m_sym, A2_sym, s2_sym, m2_sym)
/home/sam/.conda/envs/dkf/lib/python2.7/site-packages/theano/sandbox/rng_mrg.py:62: UserWarning: Warning, Cannot compute test value: input 1 (s) of Op DotModulo(A, s, m, A2, s2, m2) missing default value
  o = DotModulo()(A_sym, s_sym, m_sym, A2_sym, s2_sym, m2_sym)
/home/sam/.conda/envs/dkf/lib/python2.7/site-packages/theano/sandbox/rng_mrg.py:62: UserWarning: Warning, Cannot compute test value: input 2 (m) of Op DotModulo(A, s, m, A2, s2, m2) missing default value
  o = DotModulo()(A_sym, s_sym, m_sym, A2_sym, s2_sym, m2_sym)
/home/sam/.conda/envs/dkf/lib/python2.7/site-packages/theano/sandbox/rng_mrg.py:62: UserWarning: Warning, Cannot compute test value: input 3 (A2) of Op DotModulo(A, s, m, A2, s2, m2) missing default 

	<<EMISSION TYPE: conditional>>
	<<Modifying : [p_trans_b_cov,q_W_cov,p_emis_W_0,p_emis_W_1,p_z_W_1,p_z_W_0,q_b_st_0,q_W_input_0,q_b_input_0,p_gate_embed_b_0,p_gate_embed_b_1,W_lstm_l_0,p_emis_W_mu,p_z_b_0,p_z_b_1,p_emis_b_var,p_trans_b_mu,q_b_mu,q_W_st_0,U_lstm_l_0,b_lstm_l_0,p_emis_b_1,p_emis_b_0,p_trans_W_mu,p_emis_b_mu,q_W_mu,p_emis_W_var,q_b_cov,p_gate_embed_W_1,p_gate_embed_W_0,p_trans_W_cov]>>
<<<<<< Rescaling Gradients >>>>>>
<<<<<< Normalizing Gradients to have norm ( 1.0 ) >>>>>>
	<<0 other updates>>
	<<Building with dropout:0.0>>
	<<In <_buildLSTM>>>
	<<In _LSTMlayer with dropout 0.0000>>
	<<EMISSION TYPE: conditional>>
	<<_buildModel took : 58.7803 seconds>>


In [4]:
oodles_of_samples = dkf.sample(nsamples=50, T=1024)
sample_X, sample_Z = oodles_of_samples

In [5]:
print('Output shape: %s' % str(sample_X.shape))
mu = dataset['h36m_mean'].reshape((1, 1, -1))
sigma = dataset['h36m_std'].reshape((1, 1, -1))
real_X = insert_junk_entries(sample_X * sigma + mu)
dest_dir = './generated/'
try:
    os.makedirs(dest_dir)
except OSError:
    pass
for i, sampled_times in enumerate(real_X):
    dest_fn = os.path.join(dest_dir, 'seq-%i.txt' % i)
    print('Saving %s' % dest_fn)
    np.savetxt(dest_fn, sampled_times, delimiter=',', fmt='%f')

Output shape: (50, 1024, 54)
Saving ./generated/seq-0.txt
Saving ./generated/seq-1.txt
Saving ./generated/seq-2.txt
Saving ./generated/seq-3.txt
Saving ./generated/seq-4.txt
Saving ./generated/seq-5.txt
Saving ./generated/seq-6.txt
Saving ./generated/seq-7.txt
Saving ./generated/seq-8.txt
Saving ./generated/seq-9.txt
Saving ./generated/seq-10.txt
Saving ./generated/seq-11.txt
Saving ./generated/seq-12.txt
Saving ./generated/seq-13.txt
Saving ./generated/seq-14.txt
Saving ./generated/seq-15.txt
Saving ./generated/seq-16.txt
Saving ./generated/seq-17.txt
Saving ./generated/seq-18.txt
Saving ./generated/seq-19.txt
Saving ./generated/seq-20.txt
Saving ./generated/seq-21.txt
Saving ./generated/seq-22.txt
Saving ./generated/seq-23.txt
Saving ./generated/seq-24.txt
Saving ./generated/seq-25.txt
Saving ./generated/seq-26.txt
Saving ./generated/seq-27.txt
Saving ./generated/seq-28.txt
Saving ./generated/seq-29.txt
Saving ./generated/seq-30.txt
Saving ./generated/seq-31.txt
Saving ./generated/se

In [6]:
# Do the same thing, but smoothed
smooth_dest_dir = './generated-smooth/'
try:
    os.makedirs(smooth_dest_dir)
except OSError:
    pass
kernel = [0.1, 0.25, 0.3, 0.25, 0.1]
full_kernel = np.array(kernel).reshape((-1, 1))
for i, sampled_times in enumerate(real_X):
    dest_fn = os.path.join(smooth_dest_dir, 'seq-%i.txt' % i)
    print('Saving %s' % dest_fn)
    smooth_times = convolve2d(sampled_times, full_kernel, mode='valid')
    assert smooth_times.ndim == 2
    assert smooth_times.shape[1] == sampled_times.shape[1]
    np.savetxt(dest_fn, smooth_times, delimiter=',', fmt='%f')

Saving ./generated-smooth/seq-0.txt
Saving ./generated-smooth/seq-1.txt
Saving ./generated-smooth/seq-2.txt
Saving ./generated-smooth/seq-3.txt
Saving ./generated-smooth/seq-4.txt
Saving ./generated-smooth/seq-5.txt
Saving ./generated-smooth/seq-6.txt
Saving ./generated-smooth/seq-7.txt
Saving ./generated-smooth/seq-8.txt
Saving ./generated-smooth/seq-9.txt
Saving ./generated-smooth/seq-10.txt
Saving ./generated-smooth/seq-11.txt
Saving ./generated-smooth/seq-12.txt
Saving ./generated-smooth/seq-13.txt
Saving ./generated-smooth/seq-14.txt
Saving ./generated-smooth/seq-15.txt
Saving ./generated-smooth/seq-16.txt
Saving ./generated-smooth/seq-17.txt
Saving ./generated-smooth/seq-18.txt
Saving ./generated-smooth/seq-19.txt
Saving ./generated-smooth/seq-20.txt
Saving ./generated-smooth/seq-21.txt
Saving ./generated-smooth/seq-22.txt
Saving ./generated-smooth/seq-23.txt
Saving ./generated-smooth/seq-24.txt
Saving ./generated-smooth/seq-25.txt
Saving ./generated-smooth/seq-26.txt
Saving ./ge